In [18]:
# imports (may be more than needed)
import pandas as pd
import numpy as np
import glob # to find all files in folder
from datetime import datetime
from datetime import date, time
from dateutil.parser import parse
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import show
import itertools
%matplotlib inline
sns.set_context('notebook')
pd.options.mode.chained_assignment = None  # default='warn'


In [39]:
data = pd.read_csv('data/GrantExport.csv', delimiter=';', index_col=0)
data.dtypes


Project Title                   object
Project Title English           object
Responsible Applicant           object
Funding Instrument              object
Funding Instrument Hierarchy    object
Institution                     object
University                      object
Discipline Number                int64
Discipline Name                 object
Discipline Name Hierarchy       object
Start Date                      object
End Date                        object
Approved Amount                 object
Keywords                        object
dtype: object

In [38]:
data['Approved Amount']

﻿"Project Number"
1          11619.00
4          41022.00
5          79732.00
6          52627.00
7         120042.00
8          53009.00
9          25403.00
10         47100.00
11         25814.00
13        360000.00
14        153886.00
17        862200.00
18        116991.00
19        112664.00
20          5000.00
21        204018.00
23        149485.00
25         83983.00
26         38152.00
27         14138.00
28        164602.00
30        147795.00
32         24552.00
33         44802.00
34         56000.00
35        152535.00
36        225000.00
37        179124.00
38         20000.00
39        445198.00
            ...    
170414      1898.00
170473      9300.00
170478      9500.00
170510      4000.00
170707     11000.00
170714      7500.00
170715      8000.00
170716      9300.00
170718      7000.00
170762      6000.00
170806      4500.00
170812      2800.00
170813     99952.00
170814     98040.00
170815    100000.00
170817     10500.00
170883     10030.00
170893      7870.00
17